In [7]:
import tweepy as tw

In [2]:
consumer_key = '6caVMNywE5xqe6XE2BnNUXtef'
consumer_secret = 'JptZBnckLkPCbYqIT5ZfquQmHHTXmSoW9FNQDRekPbJBQfKBWb'
access_token = '1177416131449184258-z9iL49RwT06ynWntf4MAAW1VB1CNZO'
access_token_secret = 'oX527rbWiOfjet8aIoktAnxQ08IEpAGavMoxtYRizD8jS'

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
# collect tweets related to the phillies
search_words = "breakfast OR lunch OR dinner OR food"+ " -filter:retweets"

In [5]:
# initialize the cursor
cursor = tw.Cursor(api.search,
                   q=search_words,
                   geocode="39.8,-95.583068847656,2500km",
                   lang="en",
                   tweet_mode='extended')

In [6]:
# Iterate on tweets
for tweet in tweets:
    print(tweet.full_text)

NameError: name 'tweets' is not defined

In [66]:
tweets = [tweet for tweet in cursor.items(1000)]
# get the text of the tweets
tweets_text = [tweet.full_text for tweet in tweets]
# save the context to a csv
with open('returns.csv', 'w',encoding="utf-8") as f:
    writer = csv.writer(f)
    for word in tweets_text:
        writer.writerow([word])

In [67]:
# get the text of the tweets
tweets_text = [tweet.full_text for tweet in tweets]

In [72]:
with open('returns.csv', 'w',encoding="utf-8") as f:
    writer = csv.writer(f)
    for word in tweets_text:
        writer.writerow([word])

## Census data of NYC

In [1]:
# import the Census object
from census import Census
import pandas as pd
import geopandas as gpd

In [2]:
# initialize with your API key
api_key = '458bfa4e3d9b5cd8379b923ed09b38f62c3ffd9f'
c = Census(key=api_key)

In [3]:
import us

In [4]:
ny_code=us.states.NY.fips
ny_code

'36'

In [5]:
# TOTAL POPULATION, WHITE POPULATION,BLACK POPULATION,ASIAN POPULATION, MEDIAN HOUSEHOLD INCOME,TOTAL HOUSEHOLD, NON VEHICLE AVAILABLE HOUSEHOLD
variables=('NAME',"B01003_001E","B02001_002E","B02001_003E","B02001_005E","B19013_001E","B08201_001E","B08201_002E")
result=pd.DataFrame(c.acs5.state_county_tract(variables,"36","047","*"))#Brooklyn
result2=pd.DataFrame(c.acs5.state_county_tract(variables,"36","081","*"))#Queens

In [6]:
Brooklyn_Queens_census=result.append(result2)
Brooklyn_Queens_census.columns=['NAME','TOT_POP','WHI_POP','BLA_POP','ASI_POP','MED_INC','TOT_HOUS','NON_VEH_HOU','state','county','tract']

In [7]:
# features engineering
Brooklyn_Queens_census['prt_whi']=round((Brooklyn_Queens_census['WHI_POP']/Brooklyn_Queens_census['TOT_POP'])*100,2)
Brooklyn_Queens_census['prt_bla']=round((Brooklyn_Queens_census['BLA_POP']/Brooklyn_Queens_census['TOT_POP'])*100,2)
Brooklyn_Queens_census['prt_asi']=round((Brooklyn_Queens_census['ASI_POP']/Brooklyn_Queens_census['TOT_POP'])*100,2)
Brooklyn_Queens_census['prt_veh']=round((Brooklyn_Queens_census['NON_VEH_HOU']/Brooklyn_Queens_census['TOT_HOUS'])*100,2)

In [8]:
# get census tracts geometry
tracts = gpd.read_file(us.states.NY.shapefile_urls('tract'))
county_fp=["047","081"]
selection=tracts["COUNTYFP10"].isin(county_fp)
Brooklyn_Queens_tracts=tracts.loc[selection]
Brooklyn_Queens_tracts

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
1100,36,047,031500,36047031500,315,Census Tract 315,G5020,S,179759,0,+40.6769054,-073.9499241,"POLYGON ((-73.94708899999999 40.676557, -73.94..."
1101,36,047,031701,36047031701,317.01,Census Tract 317.01,G5020,S,139741,0,+40.6738669,-073.9501241,"POLYGON ((-73.94731299999999 40.674162, -73.94..."
1102,36,047,031702,36047031702,317.02,Census Tract 317.02,G5020,S,139786,0,+40.6711930,-073.9503750,"POLYGON ((-73.94773099999999 40.669705, -73.95..."
1103,36,047,031900,36047031900,319,Census Tract 319,G5020,S,151345,0,+40.6668852,-073.9493978,"POLYGON ((-73.94798399999999 40.667031, -73.94..."
1104,36,047,032100,36047032100,321,Census Tract 321,G5020,S,160096,0,+40.6668692,-073.9521834,"POLYGON ((-73.95075199999999 40.667181, -73.95..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3731,36,081,072300,36081072300,723,Census Tract 723,G5020,S,470056,0,+40.7162394,-073.8477446,"POLYGON ((-73.84298 40.714533, -73.84352299999..."
3732,36,081,084900,36081084900,849,Census Tract 849,G5020,S,303206,0,+40.7547095,-073.8329979,"POLYGON ((-73.83013799999999 40.753915, -73.83..."
3733,36,081,086900,36081086900,869,Census Tract 869,G5020,S,577799,25898,+40.7652626,-073.8337100,"POLYGON ((-73.835286 40.761386, -73.8353939999..."
3734,36,081,003300,36081003300,33,Census Tract 33,G5020,S,508310,0,+40.7543878,-073.9380943,"POLYGON ((-73.943609 40.752766, -73.94243 40.7..."


In [16]:
len(Brooklyn_Queens_census.loc[Brooklyn_Queens_census.TOT_POP==0])


30

In [17]:
Brooklyn_Queens=Brooklyn_Queens_tracts.merge(Brooklyn_Queens_census,left_on=["STATEFP10", "COUNTYFP10", "TRACTCE10"],
                                             right_on=["state","county","tract"],)

In [18]:
Brooklyn_Queens=Brooklyn_Queens.to_crs(epsg=3857)
Brooklyn_Queens['AREA']=Brooklyn_Queens.area
Brooklyn_Queens['pop_den']=Brooklyn_Queens['TOT_POP']/Brooklyn_Queens['AREA']

In [59]:
Brooklyn_Queens.loc[973]

STATEFP10                                                     36
COUNTYFP10                                                   047
TRACTCE10                                                 990100
GEOID10                                              36047990100
NAME10                                                      9901
NAMELSAD10                                     Census Tract 9901
MTFCC10                                                    G5020
FUNCSTAT10                                                     S
ALAND10                                                        0
AWATER10                                                17793514
INTPTLAT10                                           +40.5649933
INTPTLON10                                          -074.0148865
geometry       POLYGON ((-8238332.833503635 4952693.747105855...
NAME                   Census Tract 9901, Kings County, New York
TOT_POP                                                        0
WHI_POP                  

## Spatial infrastructure of NYC

In [21]:
import osm2gpd
import osmnx as ox
from matplotlib import pyplot as plt

In [22]:
# Brooklyn = ox.gdf_from_place('Kings County')
# Queens=ox.gdf_from_place('Queens County')

In [23]:
# county = gpd.read_file(us.states.NY.shapefile_urls('county'))
# county_name=["Kings","Queens"]
# selection=county["NAME10"].isin(county_name)
# Brooklyn_Queens_county=county.loc[selection]
# Brooklyn_Queens_county.unary_union
# Brooklyn_Queens_county

In [24]:
lng_min,lat_min,lng_max,lat_max=Brooklyn_Queens.to_crs(epsg=4326).total_bounds

In [25]:
# find the subway stations
where='station=subway'
subway1=osm2gpd.get(lng_min,lat_min,lng_max,lat_max,where=where).to_crs(epsg=3857)
subway2=gpd.sjoin(subway1.to_crs(epsg=3857),Brooklyn_Queens.to_crs(epsg=3857),op="within")
subway2["cnt"]=1

In [26]:
subways=subway2.groupby(['NAMELSAD10'])['cnt'].sum()
subways

NAMELSAD10
Census Tract 1          1
Census Tract 100        1
Census Tract 1008.02    1
Census Tract 101        1
Census Tract 106        1
                       ..
Census Tract 942.02     1
Census Tract 954        1
Census Tract 972.02     2
Census Tract 974        1
Census Tract 992        2
Name: cnt, Length: 211, dtype: int64

In [27]:
subways=subways.reset_index()
subways#['cnt'].value_counts()

,NAMELSAD10,cnt
0,Census Tract 1,1
1,Census Tract 100,1
2,Census Tract 1008.02,1
3,Census Tract 101,1
4,Census Tract 106,1
...,...,...
206,Census Tract 942.02,1
207,Census Tract 954,1
208,Census Tract 972.02,2
209,Census Tract 974,1


In [28]:
Brooklyn_Queens=Brooklyn_Queens.merge(subways,how="outer",left_on='NAMELSAD10',right_on='NAMELSAD10')
# Brooklyn_Queens[30].fillna(0, inplace=True)

In [29]:
Brooklyn_Queens1=Brooklyn_Queens.fillna(0)
Brooklyn_Queens1

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,...,state,county,tract,prt_whi,prt_bla,prt_asi,prt_veh,AREA,pop_den,cnt
0,36,047,031500,36047031500,315,Census Tract 315,G5020,S,179759,0,...,36,047,031500,29.38,56.55,3.85,83.21,3.128536e+05,0.015266,0.0
1,36,047,031701,36047031701,317.01,Census Tract 317.01,G5020,S,139741,0,...,36,047,031701,28.22,62.43,4.90,66.10,2.431847e+05,0.013595,0.0
2,36,047,031702,36047031702,317.02,Census Tract 317.02,G5020,S,139786,0,...,36,047,031702,32.22,51.22,4.37,64.62,2.432436e+05,0.015618,1.0
3,36,047,031900,36047031900,319,Census Tract 319,G5020,S,151345,0,...,36,047,031900,36.52,59.58,1.00,64.24,2.633226e+05,0.012957,1.0
4,36,047,032100,36047032100,321,Census Tract 321,G5020,S,160096,0,...,36,047,032100,24.00,67.00,3.19,73.77,2.785492e+05,0.021059,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,36,081,061301,36081061301,613.01,Census Tract 613.01,G5020,S,713922,0,...,36,081,061301,79.52,1.84,8.03,43.45,1.243697e+06,0.005018,2.0
1426,36,081,065703,36081065703,657.03,Census Tract 657.03,G5020,S,267294,0,...,36,081,065703,91.54,0.49,1.15,19.51,4.657272e+05,0.006145,0.0
1427,36,081,072300,36081072300,723,Census Tract 723,G5020,S,470056,0,...,36,081,072300,56.94,3.47,36.91,12.24,8.190441e+05,0.002322,0.0
1428,36,081,084900,36081084900,849,Census Tract 849,G5020,S,303206,0,...,36,081,084900,3.60,5.90,73.02,71.86,5.289282e+05,0.013512,0.0


In [31]:
food_desert=pd.read_csv("FoodDesert_2015.csv")
food_desert=food_desert.loc[food_desert['State']=="New York"]
food_desert=food_desert.loc[(food_desert['County']=='Queens')|(food_desert['County']=='Kings')]
food_desert

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
44152,36047000100,New York,Kings,1,4338,2227,0,424,0.097741,0,...,843,3396,427,330,0,15,170,347,1404,37
44153,36047000200,New York,Kings,1,1470,360,0,39,0.026531,0,...,70,530,103,49,0,16,772,1277,265,178
44154,36047000301,New York,Kings,1,4259,2020,0,488,0.114581,0,...,688,3658,180,241,0,6,174,246,1392,42
44155,36047000501,New York,Kings,1,4247,1879,0,1082,0.254768,0,...,553,3266,292,432,1,22,234,312,1328,103
44156,36047000502,New York,Kings,1,2512,1509,0,41,0.016322,0,...,357,2086,113,186,0,4,123,162,1016,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46845,36081157902,New York,Queens,1,3836,1107,0,68,0.017727,0,...,706,1420,125,1975,0,16,300,353,67,100
46846,36081157903,New York,Queens,1,3944,1330,0,0,0.000000,0,...,675,1903,107,1571,3,31,329,403,93,62
46847,36081161700,New York,Queens,1,4297,1468,0,14,0.003258,0,...,597,2578,218,983,0,8,510,816,152,55
46848,36081162100,New York,Queens,1,6197,1966,0,6,0.000968,0,...,579,2190,505,2429,7,28,1038,1365,202,121


In [32]:
# get the cols I want to use
col=['CensusTract','LAhalfand10']
food_desert=food_desert[col]

In [33]:
food_desert

,CensusTract,LAhalfand10
44152,36047000100,0
44153,36047000200,0
44154,36047000301,0
44155,36047000501,0
44156,36047000502,0
...,...,...
46845,36081157902,0
46846,36081157903,0
46847,36081161700,1
46848,36081162100,1


In [34]:
tracts17=Brooklyn_Queens1[['GEOID10','NAMELSAD10']]

In [35]:
import numpy as np

In [36]:
tracts17

,GEOID10,NAMELSAD10
0,36047031500,Census Tract 315
1,36047031701,Census Tract 317.01
2,36047031702,Census Tract 317.02
3,36047031900,Census Tract 319
4,36047032100,Census Tract 321
...,...,...
1425,36081061301,Census Tract 613.01
1426,36081065703,Census Tract 657.03
1427,36081072300,Census Tract 723
1428,36081084900,Census Tract 849


In [37]:
tracts17['GEOID10']=tracts17['GEOID10'].astype(np.int64)

C:\Users\xinti\AppData\Local\Continuum\anaconda3\envs\musa-620\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
tracts17['GEOID10'].dtype

dtype('int64')

In [39]:
tracts17.columns

Index(['GEOID10', 'NAMELSAD10'], dtype='object')

In [40]:
food_desert.columns


Index(['CensusTract', 'LAhalfand10'], dtype='object')

In [41]:
see1=tracts17.merge(food_desert,left_on="GEOID10",right_on='CensusTract',how='outer')

In [42]:
see1.loc[see1["LAhalfand10"].isna()==True]

,GEOID10,NAMELSAD10,CensusTract,LAhalfand10
973,36047990100,Census Tract 9901,NaN,NaN


In [43]:
see1

,GEOID10,NAMELSAD10,CensusTract,LAhalfand10
0,36047031500,Census Tract 315,3.604703e+10,0.0
1,36047031701,Census Tract 317.01,3.604703e+10,0.0
2,36047031702,Census Tract 317.02,3.604703e+10,0.0
3,36047031900,Census Tract 319,3.604703e+10,0.0
4,36047032100,Census Tract 321,3.604703e+10,0.0
...,...,...,...,...
1425,36081061301,Census Tract 613.01,3.608106e+10,0.0
1426,36081065703,Census Tract 657.03,3.608107e+10,0.0
1427,36081072300,Census Tract 723,3.608107e+10,0.0
1428,36081084900,Census Tract 849,3.608108e+10,0.0


In [44]:
Brooklyn_Queens1.columns

Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'geometry', 'NAME', 'TOT_POP', 'WHI_POP',
       'BLA_POP', 'ASI_POP', 'MED_INC', 'TOT_HOUS', 'NON_VEH_HOU', 'state',
       'county', 'tract', 'prt_whi', 'prt_bla', 'prt_asi', 'prt_veh', 'AREA',
       'pop_den', 'cnt'],
      dtype='object')

In [45]:
Brooklyn_Queens1['GEOID10']=Brooklyn_Queens1['GEOID10'].astype(np.int64)

In [46]:
Brooklyn_Queens1.GEOID10.dtype

dtype('int64')

In [47]:
BQ=Brooklyn_Queens1.merge(food_desert,left_on="GEOID10",right_on="CensusTract",how="outer")
BQ.drop('CensusTract',axis=1,inplace=True)

In [48]:
BQ.fillna(0,inplace=True)

In [49]:
newnames=BQ.columns.values
newnames[-1]='FoodDesertOrNot'
BQ.columns=newnames

In [50]:
BQ.FoodDesertOrNot=BQ.FoodDesertOrNot.astype(dtype='int32')

In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [52]:
cols=['MED_INC','prt_whi','prt_bla','prt_veh','pop_den','cnt','FoodDesertOrNot']

In [53]:
mode=BQ[cols]

In [54]:
train_set, test_set=train_test_split(mode,test_size=0.3,random_state=42)

In [55]:
y_train=train_set['FoodDesertOrNot']
y_test=test_set['FoodDesertOrNot']

feature_cols=[col for col in mode.columns if col not in ['FoodDesertOrNot']]
x_train=train_set[feature_cols].values
x_test=test_set[feature_cols].values

In [181]:
regr=RandomForestClassifier(n_estimators=10,random_state=42)

In [182]:
regr.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [183]:
y_pred=regr.predict(x_test)

In [184]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[392   4]
 [ 33   0]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       396
           1       0.00      0.00      0.00        33

    accuracy                           0.91       429
   macro avg       0.46      0.49      0.48       429
weighted avg       0.85      0.91      0.88       429

0.9137529137529138


In [185]:
BQ.columns

Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'geometry', 'NAME', 'TOT_POP', 'WHI_POP',
       'BLA_POP', 'ASI_POP', 'MED_INC', 'TOT_HOUS', 'NON_VEH_HOU', 'state',
       'county', 'tract', 'prt_whi', 'prt_bla', 'prt_asi', 'prt_veh', 'AREA',
       'pop_den', 'cnt', 'FoodDesertOrNot'],
      dtype='object')

In [189]:
BQ.FoodDesertOrNot.value_counts()

0    1311
1     119
Name: FoodDesertOrNot, dtype: int64

In [141]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

In [153]:
# X, y = make_classification(n_samples=1000, n_features=4,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)
# clf = RandomForestClassifier(max_depth=2, random_state=0)
# clf.fit(X, y)

# print(clf.feature_importances_)

# print(clf.predict([[0, 0, 0, 0]]))

[0.17287856 0.80608704 0.01884792 0.00218648]
[1]


C:\Users\xinti\AppData\Local\Continuum\anaconda3\envs\musa-620\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [57]:
# where='amenity=bus_station'
# bus=osm2gpd.get(lng_min,lat_min,lng_max,lat_max,where=where).to_crs(epsg=3857)
# bus=gpd.sjoin(bus.to_crs(epsg=3857),Brooklyn_Queens_county.to_crs(epsg=3857),op="within")
# bus

,lat,lon,amenity,bench,bus,highway,name,route_ref,shelter,addr:city,...,CLASSFP10,MTFCC10,CSAFP10,CBSAFP10,METDIVFP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
1160171467,40.609715,-73.985574,bus_station,no,yes,bus_stop,Bay Parkway & 72nd Street,B6,no,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
4678591293,40.689903,-73.951449,bus_station,NaN,NaN,NaN,Bedford-Nostrand Avs Station (G),NaN,NaN,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
4882204869,40.576718,-73.972910,bus_station,yes,yes,NaN,NaN,NaN,yes,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
6456450337,40.689583,-73.985991,bus_station,NaN,yes,NaN,Greyhound,NaN,NaN,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
3858012913,40.734398,-73.815852,bus_station,NaN,NaN,NaN,shuttle bus station & add more stops on route,NaN,NaN,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
4626264290,40.647330,-73.779678,bus_station,yes,NaN,NaN,MTA JFK Airport Terminal 5 Bus,Q3;Q10;Q10ltd;B15,yes,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
6074818985,40.710275,-73.766603,bus_station,NaN,NaN,NaN,Estación tren Holis,NaN,NaN,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
6074819085,40.699250,-73.809938,bus_station,NaN,NaN,NaN,Estación tren Jamaica,NaN,NaN,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293


In [64]:
# where='public_transport=station'
# pub=osm2gpd.get(lng_min,lat_min,lng_max,lat_max,where=where).to_crs(epsg=3857)
# pub=gpd.sjoin(pub.to_crs(epsg=3857),Brooklyn_Queens_county.to_crs(epsg=3857),op="within")
# pub

,lat,lon,image,name,network,operator,owner,public_transport,railway,start_date,...,CLASSFP10,MTFCC10,CSAFP10,CBSAFP10,METDIVFP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
42841012,40.668518,-73.751843,https://upload.wikimedia.org/wikipedia/commons...,Laurelton,Long Island Rail Road,Long Island Rail Road,Metropolitan Transit Authority,station,station,1907-04,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
274331441,40.754615,-73.845660,NaN,Mets-Willets Point,NYC Subway,New York City Transit Authority,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
276018599,40.752381,-73.844076,NaN,Mets–Willets Point,LIRR,NaN,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
276233212,40.745643,-73.902938,NaN,61st Street–Woodside Avenue,NYC Subway,New York City Transit Authority,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
276319410,40.741524,-73.957128,NaN,Long Island City,Long Island Rail Road,Long Island Railroad,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,281096933,180458283,+40.6585571,-073.8379293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6456450337,40.689583,-73.985991,NaN,Greyhound,NaN,NaN,NaN,station,NaN,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
6543642457,40.687050,-73.976209,NaN,Fulton Street,New York City Subway,New York City Transit Authority,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
6562022198,40.692206,-73.987270,NaN,Jay Street–MetroTech,New York City Subway,New York City Transit Authority,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774
6562022201,40.690604,-73.981760,NaN,DeKalb Avenue,New York City Subway,New York City Transit Authority,NaN,station,station,NaN,...,H6,G4020,408,35620,35644,C,183412016,67602622,+40.6351332,-073.9507774


In [6]:
distance=gpd.read_file("tracts_dist.geojson")

In [7]:
distance

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,...,prt_veh,AREA,pop_den,cnt,FoodDesertOrNot,fast_food,market,convenience,supermarket,geometry
0,36,047,031500,36047031500,315,Census Tract 315,G5020,S,179759,0,...,83.21,3.128536e+05,0.015266,0.0,0,192.652998,449.443702,287.644499,225.682084,POLYGON ((-8231752.293124881 4964750.318627094...
1,36,047,031701,36047031701,317.01,Census Tract 317.01,G5020,S,139741,0,...,66.10,2.431847e+05,0.013595,0.0,0,216.529167,893.731008,183.356168,315.976665,POLYGON ((-8231777.228690818 4964398.782484217...
2,36,047,031702,36047031702,317.02,Census Tract 317.02,G5020,S,139786,0,...,64.62,2.432436e+05,0.015618,1.0,0,181.437444,1217.381327,403.507331,241.827442,POLYGON ((-8231823.760237969 4963744.621280312...
3,36,047,031900,36047031900,319,Census Tract 319,G5020,S,151345,0,...,64.24,2.633226e+05,0.012957,1.0,0,209.657349,1729.896759,303.502501,788.258499,POLYGON ((-8231851.924069141 4963352.174896972...
4,36,047,032100,36047032100,321,Census Tract 321,G5020,S,160096,0,...,73.77,2.785492e+05,0.021059,0.0,0,141.054909,1511.299816,229.119548,541.235818,POLYGON ((-8232160.056419656 4963374.189049367...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,36,081,061301,36081061301,613.01,Census Tract 613.01,G5020,S,713922,0,...,43.45,1.243697e+06,0.005018,2.0,0,1064.246183,1856.388252,970.644168,1098.773454,POLYGON ((-8225844.567748482 4970018.855801529...
1426,36,081,065703,36081065703,657.03,Census Tract 657.03,G5020,S,267294,0,...,19.51,4.657272e+05,0.006145,0.0,0,1131.813311,3377.171737,1442.842243,1253.928801,POLYGON ((-8223570.755829539 4970135.317185774...
1427,36,081,072300,36081072300,723,Census Tract 723,G5020,S,470056,0,...,12.24,8.190441e+05,0.002322,0.0,0,1552.664405,4020.284590,1393.579296,698.006192,POLYGON ((-8220162.932257884 4970326.093623153...
1428,36,081,084900,36081084900,849,Census Tract 849,G5020,S,303206,0,...,71.86,5.289282e+05,0.013512,0.0,0,354.610424,864.442387,1646.399039,514.093134,POLYGON ((-8218733.367357115 4976111.662914073...
